In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("../Nexus.db")
cursor = conn.cursor()
df = pd.read_sql_query("SELECT * FROM insider_transactions", conn)
conn.close()

In [2]:
df.head()

,id,insider_role,shares_held_before,shares_acquired_or_sold,transaction_type,price_per_share,total_shares_after_transaction,salary_related,regulatory_context,notification_id
0,1,Board member of Ayfie AS and shareholder in Go...,1281943,105000,Acquisition,6.0000,1386943,0,Forordning EU 596/2014,1
1,2,"Managing Director and Shareholder in HAAS AS, ...",1758343,150000,Purchase,5.9664,1908343,0,Standard Share Purchase,2
2,3,Styremedlem,0,500000,Kjøp,4.0000,0,0,Fortrinnsrettsemisjon,3
3,4,Styremedlem,0,625000,Kjøp,4.0000,0,0,Fortrinnsrettsemisjon,3
4,5,Styremedlem,0,237500,Kjøp,4.0000,0,0,Fortrinnsrettsemisjon,3


In [3]:
df["transaction_type"].value_counts()

transaction_type
Kjøp                            3518
Acquisition                     3238
Purchase                        2915
Sale                             856
purchase                         691
                                ... 
Sale of futures contracts          1
Opened new futures contracts       1
Closed futures contracts           1
Sale of future contracts           1
Utdeling av utbytteaksjer          1
Name: count, Length: 645, dtype: int64

In [4]:
#df["transaction_type"].unique().tolist()

In [5]:
from transaction_mapping import transaction_mapping_dict

In [6]:
def transaction_mapping(transaction_type):
    return transaction_mapping_dict.get(transaction_type, 'Group Not Found')

df["transaction_type_mapped"] = df["transaction_type"].apply(transaction_mapping)

In [7]:
df["transaction_type_mapped"].value_counts()

transaction_type_mapped
Acquisition/Purchase of Shares           13567
Disposal/Sale of Shares                   1845
Grant/Award of Rights/Instruments         1720
Group Not Found                            936
Exercise/Settlement leading to Shares      611
Administrative/Other/No Transaction        359
Cash Settlement of Rights/Options          129
Acquisition of Rights/Instruments          119
Lending/Pledging/Agreement                 111
Transfer (Internal/Other)                   96
Disposal of Rights/Instruments              38
Share Buyback/Repurchase                    21
Unclassified/Needs Review                   15
Exercise and Sale (Net Disposal)            13
Transfer (Outflow/Internal)                 11
Transfer (Rights)                            9
Mixed/Multiple Transaction Type              8
Name: count, dtype: int64

In [8]:
import numpy as np

before = df["shares_held_before"]
after = df["total_shares_after_transaction"]

df["percentage_change_in_holding"] = np.where(
    (before == 0) & (after == 0),
    0,                              # case 1: 0 → 0
    np.where(
        (before == 0) & (after > 0),
        1.0,                        # case 2: 0 → positive → 100%
        (after - before) / before   # case 3: normal case
    )
)

df["percentage_change_in_holding"] = (
    df["percentage_change_in_holding"]
    .replace([np.inf, -np.inf], np.nan)
    .fillna(0)
    .clip(-3, 3)
)

In [9]:
df["volume"] = df["shares_acquired_or_sold"] * df["price_per_share"]

In [10]:
df

,id,insider_role,shares_held_before,shares_acquired_or_sold,transaction_type,price_per_share,total_shares_after_transaction,salary_related,regulatory_context,notification_id,transaction_type_mapped,percentage_change_in_holding,volume
0,1,Board member of Ayfie AS and shareholder in Go...,1281943,105000,Acquisition,6.0000,1386943,0,Forordning EU 596/2014,1,Acquisition/Purchase of Shares,0.081907,630000.00
1,2,"Managing Director and Shareholder in HAAS AS, ...",1758343,150000,Purchase,5.9664,1908343,0,Standard Share Purchase,2,Acquisition/Purchase of Shares,0.085308,894960.00
2,3,Styremedlem,0,500000,Kjøp,4.0000,0,0,Fortrinnsrettsemisjon,3,Acquisition/Purchase of Shares,0.000000,2000000.00
3,4,Styremedlem,0,625000,Kjøp,4.0000,0,0,Fortrinnsrettsemisjon,3,Acquisition/Purchase of Shares,0.000000,2500000.00
4,5,Styremedlem,0,237500,Kjøp,4.0000,0,0,Fortrinnsrettsemisjon,3,Acquisition/Purchase of Shares,0.000000,950000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19603,19604,CRO,0,700000,Sold,0.1100,0,1,Forordning EU 596/2014,6939,Disposal/Sale of Shares,0.000000,77000.00
19604,19605,CFOO,0,700000,Sold,0.1100,0,1,Forordning EU 596/2014,6939,Disposal/Sale of Shares,0.000000,77000.00
19605,19606,CPO,0,26667,Sold,0.1100,0,1,Forordning EU 596/2014,6939,Disposal/Sale of Shares,0.000000,2933.37
19606,19607,Chief Revenue Officer,0,42735,Acquisition,0.0000,42735,1,Share purchase program,6940,Acquisition/Purchase of Shares,1.000000,0.00


In [11]:
df_of_interest = df[["notification_id", "salary_related", "transaction_type_mapped", "percentage_change_in_holding", "volume"]].copy()

In [12]:
df_of_interest

,notification_id,salary_related,transaction_type_mapped,percentage_change_in_holding,volume
0,1,0,Acquisition/Purchase of Shares,0.081907,630000.00
1,2,0,Acquisition/Purchase of Shares,0.085308,894960.00
2,3,0,Acquisition/Purchase of Shares,0.000000,2000000.00
3,3,0,Acquisition/Purchase of Shares,0.000000,2500000.00
4,3,0,Acquisition/Purchase of Shares,0.000000,950000.00
...,...,...,...,...,...
19603,6939,1,Disposal/Sale of Shares,0.000000,77000.00
19604,6939,1,Disposal/Sale of Shares,0.000000,77000.00
19605,6939,1,Disposal/Sale of Shares,0.000000,2933.37
19606,6940,1,Acquisition/Purchase of Shares,1.000000,0.00


In [13]:
#Given the overall pattern of all people on this notification, what was the return?
#Taking the first value of the row as this makes for simpler cleaning and simplisety this is a weaknes.

In [14]:
df_features_agg = (
    df_of_interest
    .groupby("notification_id")
    .agg({
        "salary_related": "first",
        "transaction_type_mapped": "first",
        "percentage_change_in_holding": "mean",
        "volume": "mean",
    })
    .reset_index()
)

df_counts = (
    df_of_interest
    .groupby("notification_id")
    .size()
    .reset_index(name="n_people")
)

df_features_agg = df_features_agg.merge(df_counts, on="notification_id", how="left")

In [15]:
df_features_agg

,notification_id,salary_related,transaction_type_mapped,percentage_change_in_holding,volume,n_people
0,1,0,Acquisition/Purchase of Shares,0.081907,6.300000e+05,1
1,2,0,Acquisition/Purchase of Shares,0.085308,8.949600e+05,1
2,3,0,Acquisition/Purchase of Shares,0.000000,1.375000e+06,6
3,4,0,Disposal/Sale of Shares,1.000000,0.000000e+00,1
4,5,0,Acquisition/Purchase of Shares,0.033343,1.282238e+05,1
...,...,...,...,...,...,...
6931,6936,1,Group Not Found,1.000000,0.000000e+00,6
6932,6937,0,Acquisition/Purchase of Shares,0.000000,7.066927e+04,5
6933,6938,1,Acquisition/Purchase of Shares,1.000000,0.000000e+00,3
6934,6939,1,Disposal/Sale of Shares,0.000000,1.811333e+05,7


In [16]:
len(df_features_agg)

6936

In [ ]:
#df_features_agg.to_csv("transactions_agg.csv", index=False, encoding="utf-8")